# Sistemas Inteligentes

## Curso académico 2024-2025

### Práctica 1: Búsqueda en espacio de estados

#### Grupo 9

* Alberto Pérez Álvarez: alberto.perez25@alu.uclm.es
* M: @alu.uclm.es


## Estructura de datos
Hemos dividido las estructuras de datos en clases, una para cada entidad como se recomienda. Asi tenemos las clases de estructuras básicas Estado, Accion, Problema y Nodo. Además de las básicas, también tenemos las clases Problema, BusquedaNoInformada, BusquedaInformada, Anchura, Profundidad, PrimeroMejor y AEstrella, seguida de otra clase Heuristica para las heuristicas.

## 0.1 Importaciones
Imports necesarios para el código

In [ ]:
#Clases básicas:
import json
from math import sqrt

#Clase búsqueda
from abc import ABC, ABCMeta,abstractmethod
import time

#Clase heurística
from math import sqrt

# Busqueda Informada
from heapq import heappush, heappop # Para la PriorityQueue

## 1. Clases básicas
Consideramos básicas las clases Estado, Accion, Problema y Nodo

### 1.1. Clase Estado

In [ ]:
class Estado:
    def __init__(self, id, latitude, longitude):
        self.identifier = id
        self.latitude = latitude
        self.longitude = longitude
    def __str__(self):
        return f"Interseccion: (id={self.identifier}, latitud={self.latitude}, longitud={self.longitude})"
    def __repr__(self):
        return f"{self.identifier}"

    def __eq__(self, otro):
        if not isinstance(otro, Estado):
            return False
        else:
            return self.identifier == otro.identifier
    def __lt__(self, otro):
        return self.identifier < otro.identifier
    def esId(self, id):
        return self.identifier == id


### 1.1. Clase Accion

In [ ]:
class Accion:
    def __init__(self, origin, destination, distance, speed):
        self.origin = origin
        self.destination = destination
        # Velocidad y distancia están en distinta unidad, aunque no lo digan
        self.time = (distance/(speed*(10/36))) 
        self.distance = distance
    def __str__(self):
        return f"Calle: Origen: {self.origin}, Destino: {self.destination}, Distancia: {self.distance}, Velocidad: {self.speed})"
    def __repr__(self):
        return f"{self.origin} → {self.destination} ({self.time})"
    def __eq__(self, otro):
        if not isinstance(otro, Accion):
            return False
        else:
            return self.origin == otro.origin and self.destination == otro.destination and self.distance == otro.distance and self.speed == otro.speed


### 1.3. Clase Problema

In [ ]:
class Problema:   
    #Constructor de Problema
    def __init__(self,ruta):
        with open(ruta, 'r') as file:
            self.data = json.load(file)
        
        self.dic_estados = {}
        self.dic_acciones = {}
        self.maxSpeed = 0

        # Pasamos las intersecciones del JSON a un nuevo diccionario estados
        for inter in self.data['intersections']:
            self.dic_estados.update({inter['identifier']:Estado(inter['identifier'], inter['latitude'], inter['longitude'])})
            self.dic_acciones.update({inter['identifier']:[]})

        # Cargamos los nodos iniciales y finales del JSON
        self.Inicial = self.dic_estados[self.data["initial"]]
        self.Final = self.dic_estados[self.data["final"]]
        
        # Pasamos los segmentos del JSON a un nuevo diccionario acciones     
        for seg in self.data['segments']:
            if (seg['speed'] > self.maxSpeed):
                self.maxSpeed = seg['speed']
            self.dic_acciones[seg['origin']].append(Accion(seg['origin'], seg['destination'], seg['distance'], seg['speed']))
            
    # Obtener un objeto Estado a partir de su ID
    def getEstado(self, id):
        return self.dic_estados[id]

    # Obtener el destino de una Accion
    def getDestinoDe(self,accion):
        return self.dic_estados[accion.destination]

    # Obtener todas las acciones de un estado a partir de su ID
    def getAccionesDe(self,id):
        return self.dic_acciones[id]

### 1.4. Clase Nodo

In [ ]:
class Nodo:
    def __init__(self, interseccion, padre = None, accionTomada = None, coste = 0, profundidad = 0, nGenerado = 0):
        self.estado = interseccion
        self.padre = padre
        self.accion = accionTomada
        self.coste = coste
        self.profundidad = profundidad
        self.nGenerado = nGenerado  # El 1º generado deberia empezar en 1 no en 0, pues dijo en clase que   
                                    # se deberia contar pero ellos no lo cuentan en sus soluciones
    def __str__(self):
        return f"Nodo(estado={self.estado}, padre={self.padre}, accion={self.accion}, coste={self.coste}, profundidad={self.profundidad})"
    def __repr__(self):
        return f"{self.estado.identifier}"
    def __eq__(self,otro):
        if not isinstance(otro, Nodo):
            return False
        return self.estado.__eq__(otro.estado) and self.nGenerado.__eq__(otro.nGenerado)
    def __lt__(self,otro):
        return self.estado.__lt__(otro.estado)

## 2. Clase Busqueda

Contiene toda la lógica principal de los algoritmos de busqueda. Es la parte del código común a todos los algoritmos, por lo que es abstracta y faltan por implementar los métodos AñadirNodoAFrontera y ExtraerNodoAFrontera. Estos métodos los implementarán las clases de los tipos de algoritmos pertinentes, que heredarán de esta clase.

## 3. Algoritmos de búsqueda no informada
Para este tipo de algoritmos tenemos una clase abstracta que hereda de Búsqueda e implementa los métodos y estructuras comunes a los dos tipos de algoritmos que tenemos de búsqueda no informada (anchura y profundidad en grafos).

### 3.1 Algoritmo de anchura en grafos


### 3.2 Algoritmo de profundidad en grafos

## 4. Algoritmos de búsqueda informada

### 4.1. Primero mejor

### 4.2. A*

### 4.3. Heuristica
Para la heurística tenemos una clase dedicada abstracta que implementa los métodos comunes a ambas heurísticas y otras dos clases que implementan una de las dos heurísticas en concreto, heredando de la clase abstracta. Lo que cambia entre estas dos es simplemente la manera de calcular la distancia, entre calcularla en línea recta usando pitágoras (heurística euclidiana) o calcularla con la distancia de Manhattan.

## 5. Inicialización de las clases y determinación de la ruta
